In [133]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [134]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

### ● 不動産の一つの物件から情報を取得する。

In [ ]:
# 港区 / 品川区 / 世田谷区 / 渋谷区 / 杉並区
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13103&sc=13113&sc=13110&sc=13112&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="cassetteitem")

In [ ]:
# スクレイピングした HTML情報出力
# f = open("test.txt", "w")
# スクレイピングした HTML情報出力
with open("test.txt", "w", encoding="utf-8") as f:
    f.write(str(items))
f.close()

In [ ]:
# 各物件情報の取得
property_name     = items.find(class_="cassetteitem_content-title").get_text() if items.find(class_="cassetteitem_content-title") else None
category          = items.find(class_="cassetteitem_content-label").span.get_text() if items.find(class_="cassetteitem_content-label") else None
address           = items.find(class_="cassetteitem_detail-col1").get_text() if items.find(class_="cassetteitem_detail-col1") else None
nearest_stations  = [station.get_text() for station in items.find_all(class_="cassetteitem_detail-text")] if items.find_all(class_="cassetteitem_detail-text") else None
construction_info = items.find(class_="cassetteitem_detail-col3").find_all("div") if items.find(class_="cassetteitem_detail-col3") else None
years_since_const = construction_info[0].get_text() if construction_info and len(construction_info) > 0 else None
number_of_floors  = construction_info[1].get_text() if construction_info and len(construction_info) > 1 else None
floor_number_td   = items.select_one("tr.js-cassette_link .cassetteitem_other-col03")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None
rent_info         = items.select_one(".cassetteitem_other tbody .js-cassette_link")
rent_admin_fee    = " / ".join([item.get_text(strip=True) for item in rent_info.select(".cassetteitem_price--rent, .cassetteitem_price--administration")]) if rent_info else None
deposit_gratuity  = " / ".join([price.get_text(strip=True) for price in rent_info.select(".cassetteitem_price--deposit, .cassetteitem_price--gratuity")]) if rent_info else None
layout_total_area = " / ".join([detail.get_text(strip=True) for detail in rent_info.select(".cassetteitem_madori, .cassetteitem_menseki")]) if rent_info else None


# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("カテゴリー (Category):", category)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)
print("階数 (Floor Number):", floor_number)
print("賃料/管理費 (Rent/Administration Fee):", rent_admin_fee)
print("敷金/礼金 (Deposit/Key Money):", deposit_gratuity)
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)

In [ ]:
# 物件画像・間取り画像・詳細URLの取得
property_image_element = items.find(class_="cassetteitem_object-item")
property_image_url = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

floor_plan_image_element = items.find(class_="casssetteitem_other-thumbnail")
floor_plan_image_url = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

property_link_element = items.select_one("a[href*='/chintai/jnc_']")
property_link = "*******" + property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

# 物件画像・間取り画像・詳細URLの表示
print("物件画像 URL (Property Image URL):", property_image_url)
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)
print("物件リンク (Property Link):", property_link)

### ● 不動産の複数ページから情報を取得する。

In [135]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13103&sc=13113&sc=13110&sc=13112&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
max_page = 8  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "******" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)    


page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30
page 6 items 30
page 7 items 30
page 8 items 30


In [136]:
df = pd.DataFrame(all_data)
df = df.drop_duplicates() # 重複データの削除
df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,ＧＲＡＮ ＰＡＳＥＯ碑文谷Ｓｏｕｔｈ,賃貸マンション,東京都目黒区碑文谷３,"東急東横線/都立大学駅 歩10分, 東急大井町線/大岡山駅 歩15分,",築3年,4階建,1階,10.6万円,12000円,-,-,ワンルーム,26.08m2,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://img01.suumo.com/front/gazo/fr/bukken/4...,******/chintai/jnc_000084256966/?bc=100339644418
1,ＧＲＡＮ ＰＡＳＥＯ碑文谷Ｓｏｕｔｈ,賃貸マンション,東京都目黒区碑文谷３,"東急東横線/都立大学駅 歩10分, 東急大井町線/大岡山駅 歩15分,",築3年,4階建,2階,21万円,15000円,-,-,2LDK,48.02m2,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://img01.suumo.com/front/gazo/fr/bukken/4...,******/chintai/jnc_000084256966/?bc=100339644418


### ● Googleスプレッドシートへの書き込み・読み込み。

In [137]:
# google スプレッドシート 書き込み・読み込み
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [138]:
from dotenv import load_dotenv
import os

# 環境変数の読み込み
load_dotenv()

# 環境変数から認証情報を取得
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
PRIVATE_KEY_PATH = os.getenv("PRIVATE_KEY_PATH")

In [139]:
PRIVATE_KEY_PATH

'C:\\\\Eiko Nakamizo\\\\Teck０\\\\Step3-1\\\\nodal-height-424419-b5-4c9935cf7ad1.json'

In [140]:
# googleスプレッドシートの認証 jsonファイル読み込み(key値はGCPから取得)
SP_CREDENTIAL_FILE = PRIVATE_KEY_PATH

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    SP_CREDENTIAL_FILE,
    scopes=scopes
)
gc = gspread.authorize(credentials)


SP_SHEET_KEY = SPREADSHEET_ID # d/〇〇/edit の〇〇部分
sh  = gc.open_by_key(SP_SHEET_KEY)

In [141]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'kitasan' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, df)

In [142]:
# 不動産データの取得
SP_SHEET     = 'kitasan' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [143]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,ＧＲＡＮ ＰＡＳＥＯ碑文谷Ｓｏｕｔｈ,賃貸マンション,東京都目黒区碑文谷３,"東急東横線/都立大学駅 歩10分, 東急大井町線/大岡山駅 歩15分,",築3年,4階建,1階,10.6万円,12000円,-,-,ワンルーム,26.08m2,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://img01.suumo.com/front/gazo/fr/bukken/4...,******/chintai/jnc_000084256966/?bc=100339644418
1,ＧＲＡＮ ＰＡＳＥＯ碑文谷Ｓｏｕｔｈ,賃貸マンション,東京都目黒区碑文谷３,"東急東横線/都立大学駅 歩10分, 東急大井町線/大岡山駅 歩15分,",築3年,4階建,2階,21万円,15000円,-,-,2LDK,48.02m2,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://img01.suumo.com/front/gazo/fr/bukken/4...,******/chintai/jnc_000084256966/?bc=100339644418


### ● 不動産データを加工する。

In [144]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   名称       233 non-null    object
 1   カテゴリ     233 non-null    object
 2   アドレス     233 non-null    object
 3   アクセス     233 non-null    object
 4   築年数      233 non-null    object
 5   構造       233 non-null    object
 6   階数       233 non-null    object
 7   家賃       233 non-null    object
 8   管理費      233 non-null    object
 9   敷金       233 non-null    object
 10  礼金       233 non-null    object
 11  間取り      233 non-null    object
 12  面積       233 non-null    object
 13  物件画像URL  233 non-null    object
 14  間取画像URL  233 non-null    object
 15  物件詳細URL  233 non-null    object
dtypes: object(16)
memory usage: 29.3+ KB


In [145]:
# NaNを含む行を削除
new_df = new_df.dropna(subset=['築年数'])

# 結果の確認
print(new_df['築年数'].head(5))


0    築3年
1    築3年
2    築3年
3    築6年
4    築6年
Name: 築年数, dtype: object


In [146]:
# 築年数列のユニークな値を表示
print(new_df['築年数'].unique())


['築3年' '築6年' '築17年' '築24年' '築30年' '新築' '築2年' '築16年' '築1年' '築5年' '築12年'
 '築15年' '築13年' '築9年' '築19年' '築11年' '築4年' '築10年' '築18年']


In [147]:
def get_most_floor(x):
    if ('階建' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階建',str(x))
        list = map(int, list)
        min_value = min(list)
        return min_value

new_df['構造'] = new_df['構造'].apply(get_most_floor)
print(new_df['構造'].head(5))

0    4
1    4
2    4
3    5
4    5
Name: 構造, dtype: int64


In [148]:
def get_floor(x):
    if ('階' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = min(list)
        return min_value
    else:
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = -1*min(list)
        return min_value

new_df['階数'] = new_df['階数'].apply(get_floor)
print(new_df['階数'].head(5))

0    1
1    2
2    2
3    1
4    1
Name: 階数, dtype: int64


In [149]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

new_df['家賃'] = new_df['家賃'].apply(change_fee)
new_df['敷金'] = new_df['敷金'].apply(change_fee)
new_df['礼金'] = new_df['礼金'].apply(change_fee)

In [150]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


new_df['管理費'] = new_df['管理費'].apply(change_fee2)

In [151]:
new_df['面積'] = new_df['面積'].apply(lambda x: float(x[:-2]))

In [152]:
new_df['区'] = new_df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [153]:
new_df['市町'] = new_df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

In [154]:
def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = access.split('/')
        if len(parts) == 2:
            line_station, walk = parts
            # ' 歩'で分割できるか確認
            if ' 歩' in walk:
                station, walk_min = walk.split(' 歩')
                # 歩数の分の数値だけを抽出
                walk_min = int(re.search(r'\d+', walk_min).group())
            else:
                station = None
                walk_min = None
        else:
            line_station = access
            station = walk_min = None

        results[f'アクセス①{i}線路名'] = line_station
        results[f'アクセス①{i}駅名'] = station
        results[f'アクセス①{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
new_df = new_df.join(new_df.apply(split_access, axis=1))

In [155]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,...,市町,アクセス①1線路名,アクセス①1駅名,アクセス①1徒歩(分),アクセス①2線路名,アクセス①2駅名,アクセス①2徒歩(分),アクセス①3線路名,アクセス①3駅名,アクセス①3徒歩(分)
0,ＧＲＡＮ ＰＡＳＥＯ碑文谷Ｓｏｕｔｈ,賃貸マンション,東京都目黒区碑文谷３,"東急東横線/都立大学駅 歩10分, 東急大井町線/大岡山駅 歩15分,",築3年,4,1,10.6,12000.0,NaN,...,碑文谷,東急東横線,都立大学駅,10,東急大井町線,大岡山駅,15,,None,NaN
1,ＧＲＡＮ ＰＡＳＥＯ碑文谷Ｓｏｕｔｈ,賃貸マンション,東京都目黒区碑文谷３,"東急東横線/都立大学駅 歩10分, 東急大井町線/大岡山駅 歩15分,",築3年,4,2,21.0,15000.0,NaN,...,碑文谷,東急東横線,都立大学駅,10,東急大井町線,大岡山駅,15,,None,NaN


In [156]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   名称           233 non-null    object 
 1   カテゴリ         233 non-null    object 
 2   アドレス         233 non-null    object 
 3   アクセス         233 non-null    object 
 4   築年数          233 non-null    object 
 5   構造           233 non-null    int64  
 6   階数           233 non-null    int64  
 7   家賃           233 non-null    float64
 8   管理費          224 non-null    float64
 9   敷金           203 non-null    float64
 10  礼金           144 non-null    float64
 11  間取り          233 non-null    object 
 12  面積           233 non-null    float64
 13  物件画像URL      233 non-null    object 
 14  間取画像URL      233 non-null    object 
 15  物件詳細URL      233 non-null    object 
 16  区            233 non-null    object 
 17  市町           233 non-null    object 
 18  アクセス①1線路名    233 non-null    object 
 19  アクセス①1駅名

In [157]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'kitasan' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)

### ● 緯度経度情報取得

In [158]:
# 不動産データの取得
SP_SHEET     = 'kitasan' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [159]:
# ジオコーダーの初期化
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="your_app_name")

current_count = 0
total_count = len(new_df['アドレス'])


# 住所から緯度と経度を取得する関数
def get_lat_lon(address):
    global current_count
    current_count += 1

    try:
        location = geolocator.geocode(address)
        print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")

        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving location for address {address}: {e}")
        return None, None

In [160]:
location = geolocator.geocode("東京都目黒区上目黒３")
print(location)

テンゲージ, 10, 目黒区, 東京都, 153-0051, 日本


In [161]:
# 住所列があると仮定して、緯度経度列をデータフレームに追加
new_df['latitude'], new_df['longitude'] = zip(*new_df['アドレス'].apply(get_lat_lon))

Error retrieving location for address 東京都目黒区碑文谷３: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都目黒区碑文谷３: 'NoneType' object has no attribute 'latitude'
Error retrieving location for address 東京都目黒区碑文谷３: 'NoneType' object has no attribute 'latitude'
4/233 件目実施中 結果: 35.62125, 139.688014
5/233 件目実施中 結果: 35.62125, 139.688014
6/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
7/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
8/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
9/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
10/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
11/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
12/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
13/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
14/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
15/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
16/233 件目実施中 結果: 35.646722600000004, 139.69996555999148
17/233 件目実施中 結果: 35.646722

In [162]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'kitasan' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)

In [163]:
import os
import gspread
from google.oauth2.service_account import Credentials
from dotenv import load_dotenv
import pandas as pd
import requests
import time
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 環境変数の読み込み
load_dotenv()
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
PRIVATE_KEY_PATH = os.getenv("PRIVATE_KEY_PATH")
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

# Googleスプレッドシートの認証
credentials = Credentials.from_service_account_file(
    PRIVATE_KEY_PATH, 
    scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
)
gc = gspread.authorize(credentials)
sheet = gc.open_by_key(SPREADSHEET_ID).worksheet('kitasan')

# ログ設定
logging.basicConfig(level=logging.INFO)

# スプレッドシートからデータを読み込み
def load_data():
    return pd.DataFrame(sheet.get_all_records())

# HTTPセッションの作成とリトライ設定
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504, 522, 524])
session.mount("https://", HTTPAdapter(max_retries=retries))

# Google Maps APIから周辺施設情報を取得
def fetch_nearby_facilities(lat, lon):
    types = ['supermarket', 'hospital', 'gym', 'convenience_store']
    facilities = {}
    for place_type in types:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius=1500&type={place_type}&key={GOOGLE_MAPS_API_KEY}"
        response = session.get(url)
        if response.status_code == 200:
            results = response.json().get('results', [])
            facilities[place_type] = [result['name'] for result in results[:3]]  # 上位3つの施設名を取得
        else:
            facilities[place_type] = ["Error fetching data"]
    return facilities

# スプレッドシートに周辺施設情報を更新
def update_facilities_info(df):
    # カラム名を追加
    sheet.update_acell('AD1', 'Nearby Facilities')
    for index, row in df.iterrows():
        if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
            facilities_info = fetch_nearby_facilities(row['latitude'], row['longitude'])
            cell_label = f'AD{index + 2}'  # Assuming the column AD is where you want to write the facilities info
            try:
                sheet.update_acell(cell_label, str(facilities_info))
                logging.info(f"Updating row {index + 2} with facilities info.")
            except Exception as e:
                logging.error(f"Error updating row {index + 2}: {e}")
            time.sleep(1)  # Add delay to avoid rate limit issues

    logging.info("All facilities information has been updated successfully.")

# テスト書き込み関数
def test_write():
    try:
        sheet.update_acell('A1', 'Test Write')
        logging.info("Test write successful.")
    except Exception as e:
        logging.error(f"Test write failed: {e}")

def main():
    test_write()
    df = load_data()
    update_facilities_info(df)

if __name__ == "__main__":
    main()






INFO:root:Test write successful.
INFO:root:Updating row 2 with facilities info.
INFO:root:Updating row 3 with facilities info.
INFO:root:Updating row 4 with facilities info.
INFO:root:Updating row 5 with facilities info.
INFO:root:Updating row 6 with facilities info.
INFO:root:Updating row 7 with facilities info.
INFO:root:Updating row 8 with facilities info.
INFO:root:Updating row 9 with facilities info.
INFO:root:Updating row 10 with facilities info.
INFO:root:Updating row 11 with facilities info.
INFO:root:Updating row 12 with facilities info.
INFO:root:Updating row 13 with facilities info.
INFO:root:Updating row 14 with facilities info.
INFO:root:Updating row 15 with facilities info.
INFO:root:Updating row 16 with facilities info.
INFO:root:Updating row 17 with facilities info.
INFO:root:Updating row 18 with facilities info.
INFO:root:Updating row 19 with facilities info.
INFO:root:Updating row 20 with facilities info.
INFO:root:Updating row 21 with facilities info.
INFO:root:Updat